In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm
from multiprocessing import Pool
import numpy as np
from scipy import stats
from src.data_preprocessing.MemSum.utils import SentenceTokenizer
import nltk

nltk.download('rslp')

[nltk_data] Downloading package rslp to /home/TCU/erichm/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package rslp to /home/TCU/erichm/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [2]:
stok = nltk.data.load('tokenizers/punkt/portuguese.pickle')
stemmer = nltk.stem.RSLPStemmer() # RSLP Portuguese stemmer
# stemmer.stem("copiar") #copi
# stemmer.stem("paisagem") #pais
# stopwords = nltk.corpus.stopwords.words('portuguese')

In [3]:
train_corpus = [ json.loads(line) for line in open('rulingBR/train_v1.1.jsonl') ]
train_corpus[0].keys()
train_corpus = pd.DataFrame.from_dict(train_corpus)
train_corpus

,ementa,acordao,relatorio,voto
0,AGRAVO INTERNO EM RECURSO EXTRAORDINÁRIO. FUND...,"Vistos, relatados e discutidos estes autos, os...",O SENHOR MINISTRO ALEXANDRE DE MORAES (Relator...,O SENHOR MINISTRO ALEXANDRE DE MORAES (RELATOR...
1,DIREITO ADMINISTRATIVO. MILITAR. PROMOÇÃO. ATO...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Pree...
2,HABEAS CORPUS. CONSTITUCIONAL. PENAL. TRÁFICO ...,"Vistos, relatados e discutidos estes autos, ac...",A SENHORA MINISTRA CÁRMEN LÚCIA - (Relatora): ...,A SENHORA MINISTRA CÁRMEN LÚCIA - (Relatora): ...
3,RECLAMAÇÃO – PRETENDIDA SUBMISSÃO DO PROCESSO ...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO CELSO DE MELLO – (Relator): ...,O SENHOR MINISTRO CELSO DE MELLO – (Relator): ...
4,Habeas corpus. Penal. Crime de estelionato pra...,"HC 110.845 Vistos, relatados e discutidos este...",O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Habe...,O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Conf...
...,...,...,...,...
6368,CONFLITO NEGATIVO DE ATRIBUIÇÕES - MINISTÉRIO ...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...
6369,DIREITO CONSTITUCIONAL E TRIBUTÁRIO. PIS. MEDI...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Pree...
6370,EMBARGOS DE DECLARAÇÃO RECEBIDOS COMO AGRAVO R...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Na e...
6371,EMBARGOS DE DECLARAÇÃO NO AGRAVO DE INSTRUMENT...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO LUIZ FUX (RELATOR): Trata-se...,O SENHOR MINISTRO LUIZ FUX (RELATOR): Os embar...


In [4]:
val_corpus = [ json.loads(line) for line in open('rulingBR/val_v1.1.jsonl') ]
val_corpus[0].keys()
val_corpus = pd.DataFrame.from_dict(val_corpus)
val_corpus

,ementa,acordao,relatorio,voto
0,AGRAVO REGIMENTAL NO RECURSO EXTRAORDINÁRIO CO...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO LUIZ FUX (RELATOR): Trata-se...,O SENHOR MINISTRO LUIZ FUX (RELATOR ): A prese...
1,RECURSO EXTRAORDINÁRIO – MATÉRIA FÁTICA E LEGA...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO MARCO AURÉLIO – Em 24 de fev...,O SENHOR MINISTRO MARCO AURÉLIO (RELATOR) – Na...
2,DIREITO TRIBUTÁRIO. CONTRIBUIÇÃO SOCIAL. SALÁR...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Pree...
3,DIREITO DO TRABALHO. RESPONSABILIDADE CIVIL. A...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Pree...
4,AGRAVO REGIMENTAL EM RECURSO EXTRAORDINÁRIO. D...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO AYRES BRITTO (Relator) Trata...,O SENHOR MINISTRO AYRES BRITTO (Relator) Tenho...
...,...,...,...,...
2120,HABEAS CORPUS. PENAL. CRIME DE RACISMO. DESCLA...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...
2121,AGRAVO REGIMENTAL NO RECURSO EXTRAORDINÁRIO CO...,"Vistos, relatados e discutidos estes autos, ac...",A SENHORA MINISTRA CÁRMEN LÚCIA (Relatora): 1....,A SENHORA MINISTRA CÁRMEN LÚCIA (Relatora): 1....
2122,Terceiro agravo regimental no recurso extraord...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Luiz...,O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): O in...
2123,HABEAS CORPUS. PENAL. TRÁFICO INTERNACIONAL DE...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO TEORI ZAVASCKI (RELATOR): Tr...,O SENHOR MINISTRO TEORI ZAVASCKI (RELATOR): 1....


In [5]:
%%time
train_corpus['input_completo'] = train_corpus['relatorio'] + ' ' + train_corpus['voto']
train_corpus['text'] = [stok.tokenize(row.lower()) for row in tqdm(train_corpus['input_completo'])]
train_corpus['summary'] = [stok.tokenize(row.lower()) for row in tqdm(train_corpus['ementa'])]
train_corpus

  0%|          | 0/6373 [00:00<?, ?it/s]

  0%|          | 0/6373 [00:00<?, ?it/s]

CPU times: user 15.7 s, sys: 140 ms, total: 15.8 s
Wall time: 15.8 s


,ementa,acordao,relatorio,voto,input_completo,text,summary
0,AGRAVO INTERNO EM RECURSO EXTRAORDINÁRIO. FUND...,"Vistos, relatados e discutidos estes autos, os...",O SENHOR MINISTRO ALEXANDRE DE MORAES (Relator...,O SENHOR MINISTRO ALEXANDRE DE MORAES (RELATOR...,O SENHOR MINISTRO ALEXANDRE DE MORAES (Relator...,[o senhor ministro alexandre de moraes (relato...,"[agravo interno em recurso extraordinário., fu..."
1,DIREITO ADMINISTRATIVO. MILITAR. PROMOÇÃO. ATO...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Pree...,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,"[direito administrativo., militar., promoção.,..."
2,HABEAS CORPUS. CONSTITUCIONAL. PENAL. TRÁFICO ...,"Vistos, relatados e discutidos estes autos, ac...",A SENHORA MINISTRA CÁRMEN LÚCIA - (Relatora): ...,A SENHORA MINISTRA CÁRMEN LÚCIA - (Relatora): ...,A SENHORA MINISTRA CÁRMEN LÚCIA - (Relatora): ...,[a senhora ministra cármen lúcia - (relatora):...,"[habeas corpus., constitucional., penal., tráf..."
3,RECLAMAÇÃO – PRETENDIDA SUBMISSÃO DO PROCESSO ...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO CELSO DE MELLO – (Relator): ...,O SENHOR MINISTRO CELSO DE MELLO – (Relator): ...,O SENHOR MINISTRO CELSO DE MELLO – (Relator): ...,[o senhor ministro celso de mello – (relator):...,[reclamação – pretendida submissão do processo...
4,Habeas corpus. Penal. Crime de estelionato pra...,"HC 110.845 Vistos, relatados e discutidos este...",O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Habe...,O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Conf...,O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Habe...,[o senhor ministro dias toffoli (relator): hab...,"[habeas corpus., penal., crime de estelionato ..."
...,...,...,...,...,...,...,...
6368,CONFLITO NEGATIVO DE ATRIBUIÇÕES - MINISTÉRIO ...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,[o senhor ministro ricardo lewandowski (relato...,[conflito negativo de atribuições - ministério...
6369,DIREITO CONSTITUCIONAL E TRIBUTÁRIO. PIS. MEDI...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Pree...,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,"[direito constitucional e tributário., pis., m..."
6370,EMBARGOS DE DECLARAÇÃO RECEBIDOS COMO AGRAVO R...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Na e...,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,[embargos de declaração recebidos como agravo ...
6371,EMBARGOS DE DECLARAÇÃO NO AGRAVO DE INSTRUMENT...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO LUIZ FUX (RELATOR): Trata-se...,O SENHOR MINISTRO LUIZ FUX (RELATOR): Os embar...,O SENHOR MINISTRO LUIZ FUX (RELATOR): Trata-se...,[o senhor ministro luiz fux (relator): trata-s...,[embargos de declaração no agravo de instrumen...


In [6]:
%%time
val_corpus['input_completo'] = val_corpus['relatorio'] + ' ' + val_corpus['voto']
val_corpus['text'] = [stok.tokenize(row.lower()) for row in tqdm(val_corpus['input_completo'])]
val_corpus['summary'] = [stok.tokenize(row.lower()) for row in tqdm(val_corpus['ementa'])]
val_corpus

  0%|          | 0/2125 [00:00<?, ?it/s]

  0%|          | 0/2125 [00:00<?, ?it/s]

CPU times: user 5.27 s, sys: 63.9 ms, total: 5.33 s
Wall time: 5.32 s


,ementa,acordao,relatorio,voto,input_completo,text,summary
0,AGRAVO REGIMENTAL NO RECURSO EXTRAORDINÁRIO CO...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO LUIZ FUX (RELATOR): Trata-se...,O SENHOR MINISTRO LUIZ FUX (RELATOR ): A prese...,O SENHOR MINISTRO LUIZ FUX (RELATOR): Trata-se...,[o senhor ministro luiz fux (relator): trata-s...,[agravo regimental no recurso extraordinário c...
1,RECURSO EXTRAORDINÁRIO – MATÉRIA FÁTICA E LEGA...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO MARCO AURÉLIO – Em 24 de fev...,O SENHOR MINISTRO MARCO AURÉLIO (RELATOR) – Na...,O SENHOR MINISTRO MARCO AURÉLIO – Em 24 de fev...,[o senhor ministro marco aurélio – em 24 de fe...,[recurso extraordinário – matéria fática e leg...
2,DIREITO TRIBUTÁRIO. CONTRIBUIÇÃO SOCIAL. SALÁR...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Pree...,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,"[direito tributário., contribuição social., sa..."
3,DIREITO DO TRABALHO. RESPONSABILIDADE CIVIL. A...,"Vistos, relatados e discutidos estes autos, ac...",A Senhora Ministra Rosa Weber (Relatora): Cont...,A Senhora Ministra Rosa Weber (Relatora): Pree...,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,"[direito do trabalho., responsabilidade civil...."
4,AGRAVO REGIMENTAL EM RECURSO EXTRAORDINÁRIO. D...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO AYRES BRITTO (Relator) Trata...,O SENHOR MINISTRO AYRES BRITTO (Relator) Tenho...,O SENHOR MINISTRO AYRES BRITTO (Relator) Trata...,[o senhor ministro ayres britto (relator) trat...,"[agravo regimental em recurso extraordinário.,..."
...,...,...,...,...,...,...,...
2120,HABEAS CORPUS. PENAL. CRIME DE RACISMO. DESCLA...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,[o senhor ministro ricardo lewandowski (relato...,"[habeas corpus., penal., crime de racismo., de..."
2121,AGRAVO REGIMENTAL NO RECURSO EXTRAORDINÁRIO CO...,"Vistos, relatados e discutidos estes autos, ac...",A SENHORA MINISTRA CÁRMEN LÚCIA (Relatora): 1....,A SENHORA MINISTRA CÁRMEN LÚCIA (Relatora): 1....,A SENHORA MINISTRA CÁRMEN LÚCIA (Relatora): 1....,[a senhora ministra cármen lúcia (relatora): 1...,[agravo regimental no recurso extraordinário c...
2122,Terceiro agravo regimental no recurso extraord...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Luiz...,O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): O in...,O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Luiz...,[o senhor ministro dias toffoli (relator): lui...,[terceiro agravo regimental no recurso extraor...
2123,HABEAS CORPUS. PENAL. TRÁFICO INTERNACIONAL DE...,"Vistos, relatados e discutidos estes autos, ac...",O SENHOR MINISTRO TEORI ZAVASCKI (RELATOR): Tr...,O SENHOR MINISTRO TEORI ZAVASCKI (RELATOR): 1....,O SENHOR MINISTRO TEORI ZAVASCKI (RELATOR): Tr...,[o senhor ministro teori zavascki (relator): t...,"[habeas corpus., penal., tráfico internacional..."


In [7]:
train_corpus_final = train_corpus.drop(columns=['ementa', 'acordao', 'relatorio', 'voto'])
train_corpus_final

with open("data/custom_data/train_CUSTOM_raw.jsonl","w") as f:
    for _, row in train_corpus_final.iterrows():
        f.write(row.to_json() + "\n")

In [8]:
val_corpus_final = val_corpus.drop(columns=['ementa', 'acordao', 'relatorio', 'voto'])

with open("data/custom_data/val_CUSTOM_raw.jsonl","w") as f:
    for _, row in val_corpus_final.iterrows():
        f.write(row.to_json() + "\n")

In [9]:
def preprocess(dataframe):
    from src.data_preprocessing.MemSum.utils import greedy_extract
    index, data = dataframe
    high_rouge_episodes = greedy_extract( data['text'], data['summary'], beamsearch_size = 2 )
    indices_list = []
    scores_list  = []

    for indices, score in high_rouge_episodes:
        indices_list.append( indices )
        scores_list.append(score)

    # data["indices"] = indices_list
    # data["score"] = score_list
    return indices_list, scores_list

In [10]:
%%time
with Pool(20) as p:
    indices_scores = list(tqdm(p.imap(preprocess, train_corpus_final.iterrows()), total=len(train_corpus_final)))
    
train_corpus_final['indices'] = [valores[0] for valores in indices_scores]
train_corpus_final['score'] = [valores[1] for valores in indices_scores]

  0%|          | 0/6373 [00:00<?, ?it/s]

CPU times: user 42 s, sys: 3.47 s, total: 45.5 s
Wall time: 9min 37s


In [11]:
train_corpus_final

,input_completo,text,summary,indices,score
0,O SENHOR MINISTRO ALEXANDRE DE MORAES (Relator...,[o senhor ministro alexandre de moraes (relato...,"[agravo interno em recurso extraordinário., fu...","[[14, 11, 12, 63, 28, 13, 0, 37, 40, 52, 64], ...","[0.7472727618998294, 0.7463203463213484, 0.746..."
1,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,"[direito administrativo., militar., promoção.,...","[[83, 82, 103, 105, 104, 102, 15, 44, 39, 64],...","[0.7982266607902573, 0.7960316932879592, 0.795..."
2,A SENHORA MINISTRA CÁRMEN LÚCIA - (Relatora): ...,[a senhora ministra cármen lúcia - (relatora):...,"[habeas corpus., constitucional., penal., tráf...","[[78, 151, 89, 127, 130, 19, 126, 8, 9, 134, 2...","[0.6797366900587272, 0.6797366900587272, 0.679..."
3,O SENHOR MINISTRO CELSO DE MELLO – (Relator): ...,[o senhor ministro celso de mello – (relator):...,[reclamação – pretendida submissão do processo...,"[[13, 16, 105, 164, 121, 4, 114], [13, 16, 105...","[0.3351560719992239, 0.3351543493156124, 0.333..."
4,O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Habe...,[o senhor ministro dias toffoli (relator): hab...,"[habeas corpus., penal., crime de estelionato ...","[[34, 36, 35, 27, 26, 21, 12], [34, 36, 35, 27...","[0.718792686580177, 0.718792686580177, 0.71879..."
...,...,...,...,...,...
6368,O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,[o senhor ministro ricardo lewandowski (relato...,[conflito negativo de atribuições - ministério...,"[[30, 14, 13, 26, 17, 25, 41, 27, 28, 31], [30...","[0.8464691046668241, 0.8368932335197665, 0.832..."
6369,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,"[direito constitucional e tributário., pis., m...","[[77, 40, 78, 37, 6, 8, 12], [77, 40, 78, 37, ...","[0.5305402930412905, 0.5247621474695685, 0.522..."
6370,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,[embargos de declaração recebidos como agravo ...,"[[173, 121, 187, 103, 171, 188, 177, 175, 163,...","[0.7257569470412838, 0.7257569470412838, 0.725..."
6371,O SENHOR MINISTRO LUIZ FUX (RELATOR): Trata-se...,[o senhor ministro luiz fux (relator): trata-s...,[embargos de declaração no agravo de instrumen...,"[[33, 60, 9, 22, 13, 25, 27, 15, 11, 6, 29, 7,...","[0.7722555655645116, 0.7718649325866094, 0.771..."


In [12]:
val_corpus_final

,input_completo,text,summary
0,O SENHOR MINISTRO LUIZ FUX (RELATOR): Trata-se...,[o senhor ministro luiz fux (relator): trata-s...,[agravo regimental no recurso extraordinário c...
1,O SENHOR MINISTRO MARCO AURÉLIO – Em 24 de fev...,[o senhor ministro marco aurélio – em 24 de fe...,[recurso extraordinário – matéria fática e leg...
2,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,"[direito tributário., contribuição social., sa..."
3,A Senhora Ministra Rosa Weber (Relatora): Cont...,[a senhora ministra rosa weber (relatora): con...,"[direito do trabalho., responsabilidade civil...."
4,O SENHOR MINISTRO AYRES BRITTO (Relator) Trata...,[o senhor ministro ayres britto (relator) trat...,"[agravo regimental em recurso extraordinário.,..."
...,...,...,...
2120,O SENHOR MINISTRO RICARDO LEWANDOWSKI (RELATOR...,[o senhor ministro ricardo lewandowski (relato...,"[habeas corpus., penal., crime de racismo., de..."
2121,A SENHORA MINISTRA CÁRMEN LÚCIA (Relatora): 1....,[a senhora ministra cármen lúcia (relatora): 1...,[agravo regimental no recurso extraordinário c...
2122,O SENHOR MINISTRO DIAS TOFFOLI (RELATOR): Luiz...,[o senhor ministro dias toffoli (relator): lui...,[terceiro agravo regimental no recurso extraor...
2123,O SENHOR MINISTRO TEORI ZAVASCKI (RELATOR): Tr...,[o senhor ministro teori zavascki (relator): t...,"[habeas corpus., penal., tráfico internacional..."


In [13]:
with open("data/custom_data/train_CUSTOM_labelled.jsonl","w") as f:
    for _, row in train_corpus_final.iterrows():
        f.write(row.to_json() + "\n")

In [14]:
train_corpus['num_sentencas_texto'] = [len(row) for row in train_corpus['texto']]
train_corpus['num_sentencas_resumo'] = [len(row) for row in train_corpus['resumo']]

KeyError: 'texto'

In [ ]:
print('Entrada:')
print('Min:', np.min(train_corpus['num_sentencas_texto']), 
      'Mean:', round(np.mean(train_corpus['num_sentencas_texto']), 2), 
      'Max:', np.max(train_corpus['num_sentencas_texto']))
print('Percentil:', np.round(stats.percentileofscore(train_corpus['num_sentencas_texto'], 500), 2))
      
print('\nResumo:')
print('Min:', np.min(train_corpus['num_sentencas_resumo']), 
      'Mean:', round(np.mean(train_corpus['num_sentencas_resumo']), 2), 
      # 'Mean:', round(np.mean(train_corpus['num_sentencas_resumo']), 2), 
      'Max:', np.max(train_corpus['num_sentencas_resumo']))
print('Score:', np.round(stats.scoreatpercentile(train_corpus['num_sentencas_resumo'], 95), 2))
print('Percentil:', np.round(stats.percentileofscore(train_corpus['num_sentencas_resumo'], 183), 2))

In [ ]:
train_corpus['num_sentencas'] = [len(row) for row in train_corpus['texto']]

In [ ]:
tamanho_sentencas = []
for index, sentencas in enumerate(train_corpus['texto']):
    tamanho_sentencas_exemplo = []
    for sentenca in sentencas:
        tamanho_sentencas_exemplo += [len(sentenca)]
    
    tamanho_sentencas += [tamanho_sentencas_exemplo]